In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [6]:
import os
batch_size = 32
image_size = 64

load_dir = '/content/drive/MyDrive/dataset/processed_dogs/'

for _, d, f in os.walk(load_dir+'all-dogs'):
  print(len(f))


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

loaded_dataset = datasets.ImageFolder(load_dir, transform=transform)

train_loader = DataLoader(loaded_dataset, batch_size=batch_size, shuffle=True)


20579


In [7]:
loaded_dataset

Dataset ImageFolder
    Number of datapoints: 20579
    Root location: /content/drive/MyDrive/dataset/processed_dogs/
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [8]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, z_dim=100, img_channels=3):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.gen(input)


In [9]:
class Discriminator(nn.Module):
    def __init__(self, img_channels=3):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.disc(input).view(-1, 1).squeeze(1)


In [10]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


In [11]:
import torch.optim as optim

lr = 0.0003
beta1 = 0.5
epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

netG = Generator().to(device)
netD = Discriminator().to(device)
netG.apply(weights_init)
netD.apply(weights_init)

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Loss function
criterion = nn.BCELoss()

In [12]:
import numpy as np
import matplotlib.pyplot as plt


def show_images(image_tensor, num_images=8):
    image_tensor = (image_tensor + 1) / 2
    image_tensor = image_tensor.clamp(0, 1)
    images = image_tensor.cpu().numpy()

    fig, axes = plt.subplots(1, num_images, figsize=(num_images*3, 3))
    for i, ax in enumerate(axes):
        ax.imshow(np.transpose(images[i], (1, 2, 0)), interpolation='nearest')
        ax.axis('off')
    plt.show()

In [13]:
import torchvision.utils as vutils
import os
# Lists to keep track of progress
img_list = []

num_test_samples = 64
fixed_noise = torch.randn(num_test_samples, 100, 1, 1, device=device)


checkpoint_dir = '/content/drive/MyDrive/gan_checkpoint/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    print(f"Created directory: {checkpoint_dir}")
else:
    print(f"Directory already exists: {checkpoint_dir}")


Directory already exists: /content/drive/MyDrive/gan_checkpoint/


In [ ]:
for epoch in range(epochs):
    for i, data in enumerate(train_loader, 0):

        netD.zero_grad()

        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), 1, dtype=torch.float, device=device)
        output = netD(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, 100, 1, 1, device=device)
        fake = netG(noise)
        label.fill_(0)
        output = netD(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        netG.zero_grad()
        label.fill_(1)
        output = netD(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, epochs, i, len(train_loader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

    if (epoch % 1 == 0) or ((epoch == epochs-1) and (i == len(train_loader)-1)):
        with torch.no_grad():
            fake = netG(fixed_noise).detach().cpu()
        img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        show_images(fake, num_images=8)

        torch.save(netG.state_dict(), checkpoint_dir + 'netG.pth')
        torch.save(netD.state_dict(), checkpoint_dir + 'netD.pth')


print("Training Finished.")


[0/1][0/644]	Loss_D: 0.5423	Loss_G: 8.3487	D(x): 0.8262	D(G(z)): 0.2368 / 0.0003
